<a href="https://colab.research.google.com/github/sujanshahi050/stepcount_dashboard/blob/main/Fitness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Data

In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

# Parse the XML file
tree = ET.parse('/content/drive/MyDrive/apple_health_export/export.xml')
root = tree.getroot()

# Extract step count records
step_count_records = []

for record in root.findall('.//Record[@type="HKQuantityTypeIdentifierStepCount"]'):
  source_name = record.attrib.get('sourceName', '')
  creation_date = record.attrib.get('creationDate', '')
  start_date = record.attrib.get('startDate', '')
  end_date = record.attrib.get('endDate', '')
  value = record.attrib.get('value', '')

  step_count_records.append({
      'SourceName': source_name,
      'CreationDate': creation_date,
      'StartDate': start_date,
      'EndDate': end_date,
      'StepCount': int(value) if value.isdigit() else None
  })

# Create a DataFrame
df = pd.DataFrame(step_count_records)

# Convert date columns to datetime format
df['CreationDate'] = pd.to_datetime(df['CreationDate'])
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])




# Data Exploration

In [ ]:
# Print top 5 rows
df.head()

,SourceName,CreationDate,StartDate,EndDate,StepCount
0,Iphone,2018-12-18 13:56:21-07:00,2018-12-18 13:16:25-07:00,2018-12-18 13:18:15-07:00,138
1,Iphone,2018-12-18 13:56:21-07:00,2018-12-18 13:18:15-07:00,2018-12-18 13:53:50-07:00,5
2,Iphone,2018-12-18 14:23:33-07:00,2018-12-18 13:53:50-07:00,2018-12-18 13:53:50-07:00,14
3,Iphone,2018-12-20 10:53:05-07:00,2018-12-20 10:42:04-07:00,2018-12-20 10:47:28-07:00,37
4,Iphone,2018-12-20 11:24:03-07:00,2018-12-20 10:47:28-07:00,2018-12-20 11:22:50-07:00,13


# Data Transformation

It looks like the data for step counts are not aggreagated for each single day and are instead separated for each session in a day where a session is the difference between the StartDate and EndDate. So, let's go ahead and aggregate the data.

In [ ]:
# Extracting Month and Day from 'CreationDate'
df['Year'] = df['CreationDate'].dt.year
df['Month'] = df['CreationDate'].dt.month
df['Day'] = df['CreationDate'].dt.day

# Calculate total hours for each session in hours
df['DurationHours'] = (df['EndDate'] - df['StartDate']).dt.total_seconds() / 3600
df.head()

,SourceName,CreationDate,StartDate,EndDate,StepCount,Year,Month,Day,DurationHours
0,Iphone,2018-12-18 13:56:21-07:00,2018-12-18 13:16:25-07:00,2018-12-18 13:18:15-07:00,138,2018,12,18,0.030556
1,Iphone,2018-12-18 13:56:21-07:00,2018-12-18 13:18:15-07:00,2018-12-18 13:53:50-07:00,5,2018,12,18,0.593056
2,Iphone,2018-12-18 14:23:33-07:00,2018-12-18 13:53:50-07:00,2018-12-18 13:53:50-07:00,14,2018,12,18,0.000000
3,Iphone,2018-12-20 10:53:05-07:00,2018-12-20 10:42:04-07:00,2018-12-20 10:47:28-07:00,37,2018,12,20,0.090000
4,Iphone,2018-12-20 11:24:03-07:00,2018-12-20 10:47:28-07:00,2018-12-20 11:22:50-07:00,13,2018,12,20,0.589444


Since we are only looking at the data for the year 2023, let's go ahead and just select the data for 2023 and store that data in a dataframe.

In [ ]:
# Select the data for 2023
year2023_data = df[df['CreationDate'].dt.year == 2023]

# Let's aggregate step counts and hours walked for each day
year2023_data_filtered = year2023_data.groupby(['Month', 'Day']).agg({'StepCount': 'sum', 'DurationHours': 'sum'}).reset_index()


In [ ]:
# Display Data
year2023_data_filtered.head()

,Month,Day,StepCount,DurationHours
0,1,1,7833,3.265556
1,1,2,6410,3.309167
2,1,3,7497,3.887778
3,1,4,3057,2.061667
4,1,5,7057,2.795278


# Exporting Data to a csv file

In [ ]:
from google.colab import files

year2023_data_filtered.to_csv('stepcount_data_2023.csv', index=False)
files.download('stepcount_data_2023.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>